In [1]:
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score
from kaggle_secrets import UserSecretsClient
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
import wandb
import torch
import spacy
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from spacy import displacy
nlp = spacy.blank('en')
from transformers import BertTokenizer
from torch.utils.data import TensorDataset
from transformers import BertForSequenceClassification

from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
try:
    user_secrets = UserSecretsClient()
    secret_value_0 = user_secrets.get_secret("wandb_api")
    wandb.login(key=secret_value_0)
    anony=None
except:
    anony = "must"
    
wandb.init(project="PICO Evidence Summarization",name=f"15.PICO Evidence Summarization BioBert-Base-Cased-v1.2-Run 15")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
df=pd.read_csv('/kaggle/input/biobert-custom-dataset/converted_input.csv')
print(df.head())
df_custom=pd.read_csv('/kaggle/input/biobert-custom-dataset/PICO_annotated_input_custom.csv')
print(df_custom.head())
df_custom.rename(columns={'ner_tags': 'Tags'}, inplace=True)
df.rename(columns={'ner_tags': 'Tags'}, inplace=True)
df=df.drop('Unnamed: 0', axis=1)
df.rename(columns={'tokens': 'Sentences'}, inplace=True)
df_custom.rename(columns={'tokens': 'Sentences'}, inplace=True)
df_custom=df_custom.drop('Unnamed: 0', axis=1)

   Unnamed: 0                                             tokens      ner_tags
0           0  To test the efficacy of a pregnancy adapted ve...             O
1           1                             Online and telephone .             O
2           2  Self-referred pregnant women ( gestational wee...  I-POPULATION
3           3  @ pregnant women ( gestational week @ ) with m...             O
4           4  The primary outcome was depressive symptoms me...     I-OUTCOME
   Unnamed: 0                                             tokens  \
0           0  on a reduction in the incidence of childhood o...   
1           1                           In school-age children .   
2           2  what is the effect of a school-based physical ...   
3           3                    compared with no intervention .   
4           4  on a reduction in reported incidences of bully...   

         ner_tags  
0               O  
1   I-PARTICIPANT  
2  I-INTERVENTION  
3  I-INTERVENTION  
4               O

In [4]:
print(df['Tags'].value_counts())
print(df_custom['Tags'].value_counts())

O                 257118
I-INTERVENTION     24602
I-OUTCOME          24094
I-PARTICIPANT      12239
I-POPULATION        1915
Name: Tags, dtype: int64
I-INTERVENTION    24
O                 12
I-PARTICIPANT     12
Name: Tags, dtype: int64


In [5]:
evidence_labels = df.Tags.unique()

evidence_dict = {}
for index, evidence_labels in enumerate(evidence_labels):
    evidence_dict[evidence_labels] = index
evidence_dict

{'O': 0,
 'I-POPULATION': 1,
 'I-OUTCOME': 2,
 'I-PARTICIPANT': 3,
 'I-INTERVENTION': 4}

In [6]:
df['Tags Mapped'] = df.Tags.map(evidence_dict)
print(df)
df_custom['Tags Mapped'] = df_custom.Tags.map(evidence_dict)
print(df_custom)

                                                Sentences          Tags  \
0       To test the efficacy of a pregnancy adapted ve...             O   
1                                  Online and telephone .             O   
2       Self-referred pregnant women ( gestational wee...  I-POPULATION   
3       @ pregnant women ( gestational week @ ) with m...             O   
4       The primary outcome was depressive symptoms me...     I-OUTCOME   
...                                                   ...           ...   
319963  After tumor progression , @ % of the systemic ...             O   
319964  Toxicity included ulcer disease ( @ % ) and bi...             O   
319965  Extrahepatic disease occurred in @ % and @ % o...             O   
319966  The median survivals were @ and @ months , for...             O   
319967  When compared with systemic therapy , hepatic ...             O   

        Tags Mapped  
0                 0  
1                 0  
2                 1  
3          

In [7]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values,\
                                                  df['Tags Mapped'].values,\
                                                  test_size=0.15,\
                                                  random_state=42,\
                                                  stratify=df['Tags Mapped'].values)

In [8]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'


# df_custom['data_type'] = ['not_set']*df_custom.shape[0]

# df_custom.loc[df_custom, 'data_type'] = 'val'

In [9]:
df.groupby(['Tags', 'Tags Mapped', 'data_type']).count()
df_custom.groupby(['Tags', 'Tags Mapped']).count()

,,Sentences
Tags,Tags Mapped,
I-INTERVENTION,4,24
I-PARTICIPANT,3,12
O,0,12


In [10]:
print(df.head())
print(df_custom.head())

                                           Sentences          Tags  \
0  To test the efficacy of a pregnancy adapted ve...             O   
1                             Online and telephone .             O   
2  Self-referred pregnant women ( gestational wee...  I-POPULATION   
3  @ pregnant women ( gestational week @ ) with m...             O   
4  The primary outcome was depressive symptoms me...     I-OUTCOME   

   Tags Mapped data_type  
0            0     train  
1            0     train  
2            1     train  
3            0     train  
4            2     train  
                                           Sentences            Tags  \
0  on a reduction in the incidence of childhood o...               O   
1                           In school-age children .   I-PARTICIPANT   
2  what is the effect of a school-based physical ...  I-INTERVENTION   
3                    compared with no intervention .  I-INTERVENTION   
4  on a reduction in reported incidences of bully...     

In [11]:
tokenizer = BertTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', do_lower_case=True,truncation=True)

In [12]:
df.shape

(319968, 4)

In [13]:
df= df.iloc[200000:,:]

In [14]:
df.shape

(119968, 4)

In [15]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Sentences.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    truncation=True,
    padding='longest', 
    max_length=512, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Sentences.values, 
    add_special_tokens=True, 
    truncation=True,
    return_attention_mask=True, 
    padding='longest', 
    max_length=512, 
    return_tensors='pt'
)

encoded_data_val_custom = tokenizer.batch_encode_plus(
    df_custom.Sentences.values, 
    add_special_tokens=True, 
    truncation=True,
    return_attention_mask=True, 
    padding='longest', 
    max_length=512, 
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
tags_train = torch.tensor(df[df.data_type=='train']['Tags Mapped'].values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
tags_val = torch.tensor(df[df.data_type=='val']['Tags Mapped'].values)

input_ids_val_custom = encoded_data_val_custom['input_ids']
attention_masks_val_custom = encoded_data_val_custom['attention_mask']
tags_val_custom = torch.tensor(df_custom['Tags Mapped'].values)

In [16]:
input_ids_train.shape

torch.Size([102061, 459])

In [17]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, tags_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, tags_val)
dataset_val_custom = TensorDataset(input_ids_val_custom, attention_masks_val_custom, tags_val_custom)

In [18]:
print(len(dataset_train), len(dataset_val), len(dataset_val_custom))

102061 17907 48


In [19]:
model = BertForSequenceClassification.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
                                                      num_labels=len(evidence_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Ber

In [20]:
batch_size = 16 #Defining Batch Size on which model has to be trained

# Converting tensors Dataset to dataloaders so that model can be trained
dataloader_train = DataLoader(dataset_train,sampler=RandomSampler(dataset_train), batch_size=batch_size) 

dataloader_validation = DataLoader(dataset_val, sampler=SequentialSampler(dataset_val), batch_size=batch_size)

dataloader_validation_custom = DataLoader(dataset_val_custom, sampler=SequentialSampler(dataset_val_custom), batch_size=batch_size)

In [21]:
optimizer = AdamW(model.parameters(),lr=8e-7, eps=1e-8)

In [22]:
epochs = 3
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,num_training_steps=len(dataloader_train)*epochs)

In [23]:
def f1_score_func(preds, tags):
    preds_flat = np.argmax(preds, axis=1).flatten()
    tags_flat = tags.flatten()
    return f1_score(tags_flat, preds_flat, average='weighted')

def recall_func(preds, tags):
    preds_flat = np.argmax(preds, axis=1).flatten()
    tags_flat = tags.flatten()
    return recall_score(tags_flat, preds_flat, average='weighted')

def accuracy_func(preds, tags):
    preds_flat = np.argmax(preds, axis=1).flatten()
    tags_flat = tags.flatten()
    return accuracy_score(tags_flat, preds_flat)

def precision_func(preds, tags):
    preds_flat = np.argmax(preds, axis=1).flatten()
    tags_flat = tags.flatten()
    return precision_score(tags_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, tags):
    evidence_dict_inverse = {v: k for k, v in evidence_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    tags_flat = tags.flatten()

    for tag in np.unique(tags_flat):
        y_preds = preds_flat[tags_flat==tag]
        y_true = tags_flat[tags_flat==tag]
        print(f'PICO Evidence Class: {evidence_dict_inverse[tag]}')
        print(f'Accuracy: {len(y_preds[y_preds==tag])}/{len(y_true)}\n')

In [24]:
seed_val = 199
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [25]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [26]:
!mkdir models

In [27]:
loss_train_avg_list = []
val_loss_list = []
val_f1_list = []

for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
    torch.save(model.state_dict(), f'models/finetuned_BioBERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    loss_train_avg_list.append(loss_train_avg)
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    val_loss_list.append(val_loss)
    tqdm.write(f'F1 validation Score (Weighted): {val_f1}')
    val_f1_list.append(val_f1)
    wandb.log({"Training loss":loss_train_avg, "Validation loss":val_loss,"F1 validation Score (Weighted)":val_f1,})

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/6379 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.3831699458052359
Validation loss: 0.3082988997189594
F1 validation Score (Weighted): 0.8696132924756211


Epoch 2:   0%|          | 0/6379 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.2969108653073915
Validation loss: 0.2987655345196669
F1 validation Score (Weighted): 0.8786091429470055


Epoch 3:   0%|          | 0/6379 [00:00<?, ?it/s]


Epoch 3
Training loss: 0.28948483456676316
Validation loss: 0.2974230777218638
F1 validation Score (Weighted): 0.8802952944212314


In [29]:
model.load_state_dict(torch.load('models/finetuned_BioBERT_epoch_3.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [30]:
wandb.finish()

F1 validation Score (Weighted),▁▇█
Training loss,█▂▁
Validation loss,█▂▁
F1 validation Score (Weighted),0.8803
Training loss,0.28948
Validation loss,0.29742


In [31]:
_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

PICO Evidence Class: O
Accuracy: 13872/14633

PICO Evidence Class: I-POPULATION
Accuracy: 0/91

PICO Evidence Class: I-OUTCOME
Accuracy: 864/1192

PICO Evidence Class: I-PARTICIPANT
Accuracy: 91/529

PICO Evidence Class: I-INTERVENTION
Accuracy: 1090/1462



In [32]:
val_loss_custom, predictions_custom, true_vals_custom = evaluate(dataloader_validation_custom)
accuracy_per_class(predictions_custom, true_vals_custom)

PICO Evidence Class: O
Accuracy: 11/12

PICO Evidence Class: I-PARTICIPANT
Accuracy: 0/12

PICO Evidence Class: I-INTERVENTION
Accuracy: 14/24



In [33]:
val_f1_custom = f1_score_func(predictions_custom, true_vals_custom)
tqdm.write(f'F1 custom validation Score (Weighted): {val_f1_custom}')

val_recall_custom = recall_func(predictions_custom, true_vals_custom)
tqdm.write(f'Recall custom validation Score (Weighted): {val_recall_custom}')

val_accuracy_custom = accuracy_func(predictions_custom, true_vals_custom)
tqdm.write(f'Accuracy custom validation Score (Weighted): {val_accuracy_custom}')

val_precision_custom = precision_func(predictions_custom, true_vals_custom)
tqdm.write(f'Precision custom validation Score (Weighted): {val_precision_custom}')

F1 custom validation Score (Weighted): 0.4934210526315789
Recall custom validation Score (Weighted): 0.5208333333333334
Accuracy custom validation Score (Weighted): 0.5208333333333334
Precision custom validation Score (Weighted): 0.5859375


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
